In [1]:
print("hello")

hello


In [2]:
from bs4 import BeautifulSoup

In [4]:
with open('index.html', 'r') as f:
    doc = BeautifulSoup(f, 'html.parser')
    # print(doc.prettify())
    # data = doc.find_all('h1')
    data = doc.find_all('p')
    print(data[0].text)
    # print(data)

Lorem ipsum dolor sit amet consectetur adipisicing elit. Quibusdam deleniti in officiis adipisci iusto nostrum quam impedit fugit, veniam ea.


In [5]:
import requests

In [6]:
url = 'https:google.com'
response = requests.get(url)
print(response.status_code)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    with open("cort.html", 'w') as f:
        f.write(response.text)
    print(soup.prettify())

InvalidURL: Invalid URL 'https:google.com': No host supplied

In [48]:
import pandas as pd

In [49]:
with open('onlinekhabar.html', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    news = []
    for tag in soup.find_all(['h2', 'h3']):
        a = tag.find("a")
        if a and a.get('href'):
            title = a.get_text(strip=True)
            link = a['href']
            if title:
                news.append({
                    'headline': title,
                    'link': link
                })
    
    df = pd.DataFrame(news)
    df = df.drop_duplicates(subset='headline').reset_index(drop = True)
    df.to_csv("links.csv", index=False)

In [55]:
import pandas as pd

In [67]:
base_url = "https://www.infinitecourses.org"
url="https://www.infinitecourses.org/courses/page-2"
rows = []

for i in range(1,16):
    response = requests.get(f"https://www.infinitecourses.org/courses/page-{i}")
    print(f"Scraping page {i}")
    print(response.status_code)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        nav = soup.find("nav")
        div = nav.find_next_sibling("div").find_next("div")
        all_a = div.find_all("a")
        for anchor in all_a:
            slug = anchor['href']
            thumbnail = anchor.find_next('img')['src']
            p4_div = anchor.find_next("div", class_="p-4")
            title = p4_div.find_next("p").text

            tag_divs = p4_div.find_next("div").find_all("div")
            created_at = tag_divs[-1].text
            tags = []
            for i in range(len(tag_divs)):
                if i < len(tag_divs)-1:
                    tags.append(tag_divs[i].text)
            
            inside_response = requests.get(f"https://www.infinitecourses.org{slug}")
            if inside_response.status_code == 200:
                new_soup = BeautifulSoup(inside_response.text, 'html.parser')

                learn_h2 = new_soup.find("h2", string="What you'll learn")
                if learn_h2:
                    learn_ul = learn_h2.find_next("ul")
                    if not learn_ul:
                        continue
                    learn = [li.text for li in learn_ul.find_all("li")]

                requirements_h2 = new_soup.find("h2", string="Requirements")
                if requirements_h2:
                    requirements_ul = requirements_h2.find_next("ul")
                    requirements = [li.text for li in requirements_ul.find_all("li")]

                description_h2 = new_soup.find("h2", string="Description")
                if description_h2:
                    descriptions = []
                    for sibling in description_h2.find_next_siblings():
                        if sibling.name == "h2":
                            break
                        elif sibling.name == "ul":
                            [descriptions.append(li.text) for li in sibling.find_all("li")]
                        else:
                            descriptions.append(sibling.text)
                
                client_h2 = new_soup.find("h2", string="Who this course is for:")
                if client_h2:
                    client_ul = client_h2.find_next("ul")
                    clients = [li.text for li in client_ul.find_all("li")]
            else:
                print("Error in inside response")
                print(inside_response.status_code)

            rows.append({
                "Title": title,
                "Thumbnail": thumbnail,
                "Slug": slug,
                "Tags": tags or [],
                "Date": created_at,
                "Description": descriptions or [],
                "Requirements": requirements or [],
                "What You Will Learn": learn or [],
                "Who This Course Is For": clients or []
            })

df = pd.DataFrame(rows)
df.to_csv("courses.csv")
df.head()
        



Scraping page 1
200
Scraping page 2
200
Scraping page 3
200
Scraping page 4
200
Scraping page 5
200
Scraping page 6
200
Scraping page 7
200
Scraping page 8
200
Scraping page 9
200
Scraping page 10
200
Scraping page 11
200
Scraping page 12
200
Scraping page 13
200
Scraping page 14
200
Scraping page 15
200


,Title,Thumbnail,Slug,Tags,Date,Description,Requirements,What You Will Learn,Who This Course Is For
0,Udemy - Playwright JS/TS Automation Testing fr...,https://cdn.sanity.io/images/mfatcmw0/producti...,/course/udemy-playwright-js-ts-automation-testing,"[udemy, javascript, typescript]","Jan 29, 2026",[**Course last updated on Jan 2026 with Playwr...,[No Programming experience required. JavaScrip...,[Learn JavaScript & TypeScript fundamentals re...,"[Automation testers, QA, Developers]"
1,Udemy - Postman: The Complete Guide - REST API...,https://cdn.sanity.io/images/mfatcmw0/producti...,/course/udemy-postman-the-complete-guide,"[udemy, postman, rest api]","Jan 29, 2026","[Learn Postman with a Postman Supernova., REST...",[Registering to a few reputable public APIs is...,"[Create GET, POST, PUT, DELETE requests, Under...",[This course is targeted towards testing engin...
2,"Udemy - GitLab CI/CD: Pipelines, CI/CD and Dev...",https://cdn.sanity.io/images/mfatcmw0/producti...,/course/udemy-gitlab-ci-cd,"[udemy, docker, aws, devops ]","Jan 28, 2026","[This course is neither endorsed by, nor in pa...",[Registering for a GitLab .com account (the fr...,"[What is a pipeline, What is Continuous Integr...",[Software developers learning to build pipelin...
3,Udemy - Learn JMETER from Scratch on Live Apps...,https://cdn.sanity.io/images/mfatcmw0/producti...,/course/udemy-learn-jmeter-from-scratch-on-liv...,"[udemy, jmeter, selenium, rest api]","Jan 28, 2026",[UPDATE: *Course is fully updated in January w...,[******You need NOT have any prior load testin...,[At the end of this Course you will be able to...,"[Manual testers, Webservice Testers,Non-progra..."
4,Udemy - HashiCorp Certified: Terraform Associa...,https://cdn.sanity.io/images/mfatcmw0/producti...,/course/udemy-hashicorp-certified-terraform-as...,[udemy],"Jan 28, 2026",[With tremendous popularity and adoption of Te...,"[Basic Understanding of AWS, AWS Account]",[PASS the HashiCorp Certified Terraform Associ...,[Individuals who are interested in mastering T...


In [69]:
df.to_csv("courses.csv", index=False)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   281 non-null    object
 1   Thumbnail               281 non-null    object
 2   Slug                    281 non-null    object
 3   Tags                    281 non-null    object
 4   Date                    281 non-null    object
 5   Description             281 non-null    object
 6   Requirements            281 non-null    object
 7   What You Will Learn     281 non-null    object
 8   Who This Course Is For  281 non-null    object
dtypes: object(9)
memory usage: 19.9+ KB


In [ ]:
import re


inside_response = requests.get("https://www.infinitecourses.org/course/udemy-the-ultimate-react-course-react-next-js-redux-and-more")
if inside_response.status_code == 200:
    new_soup = BeautifulSoup(inside_response.text, 'html.parser')
    learn_ul = new_soup.find("h2", string="What you'll learn").find_next("ul")
    learn = [li.text for li in learn_ul.find_all("li")]

    requirements_ul = new_soup.find("h2", string="Requirements").find_next("ul")
    requirements = [li.text for li in requirements_ul.find_all("li")]

    description_h2 = new_soup.find("h2", string="Description")
    descriptions = []
    for sibling in description_h2.find_next_siblings():
        if sibling.name == "h2":
            break
        elif sibling.name == "ul":
            [descriptions.append(li.text) for li in sibling.find_all("li")]
        else:
            descriptions.append(sibling.text)
    
    client_ul = new_soup.find("h2", string="Who this course is for:").find_next("ul")
    clients = [li.text for li in client_ul.find_all("li")]
else:
    print("Error in inside response")
    print(inside_response.status_code)

[<li class="my-2">Contains <strong class="font-normal">the biggest and most professional project</strong>, where we implement many common app features (see promo video!)</li>, <li class="my-2"><strong class="font-normal">Super detailed explanations</strong> of all important concepts with carefully crafted and animated diagrams</li>, <li class="my-2">A huge focus on "<strong class="font-normal">how to think in React</strong>" and <strong class="font-normal">modern best practices</strong></li>, <li class="my-2">A long section that explores <strong class="font-normal">how React actually works behind the scenes</strong> will give you the confidence to use React effectively on your own</li>, <li class="my-2"><strong class="font-normal">Advanced design patterns</strong> like compound components will make you think like a senior React engineer</li>, <li class="my-2">Many sections (16 hours!) on <strong class="font-normal">Next.js with the modern "app" router</strong> (with React Server Compon